<h1 style="text-align:center">Análise Exploratória de Dados de Notícias de Jornal Web</h1>

<div align="center"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/8f/Correio_Braziliense.svg/2560px-Correio_Braziliense.svg.png" width="500px" height="50px" alt="Logo Correio Braziliense"></img></div>

<p align='justify'>As notícias de jornal web são coletadas diariamente por webscraping a partir do site de notícias <a href="https://www.correiobraziliense.com.br/">Correio Braziliense</a>. É um jornal brasileiro com sede em Brasília - DF, fundado em 21 de abril de 1960 por Assis Chateaubriand, juntamente com a inauguração da cidade e da TV Brasília. O nome veio do histórico <a href="https://pt.wikipedia.org/wiki/Correio_Braziliense_(1808)">Correio Brasiliense ou Armazém Literário</a>, editado em Londres a partir de 1808 por Hipólito José da Costa <cite><a href="https://pt.wikipedia.org/wiki/Correio_Braziliense">[Wikipedia]</a></cite>.</p>

<p align='justify'>Como premissa para as análises e organização do projeto, é utilizada a metodologia <a href="https://www.datascience-pm.com/crisp-dm-2/">Cross Industry Standard Process for Data Mining (CRISP-DM)</a>.</p>

<h2 style="text-align:left">Entendimento do Negócio</h2>

<p align='justify'>O conjunto de dados em análise, refere-se a notícias diárias de um jornal de Brasília-DF chamado Correio Braziliense, onde por meio da técnica de webscraping, é coletado as notícias diárias desse jornal, uma tabela dispõe de informações das URL's das notícias, tema, e data da extração, a outra tabela dispõe de informações da notícias em si, data da publicação, autor, título e o texto da notícias.</p>

<h3 style="text-align:left">Dicionário de dados</h3>
<p align='justify'>A seguir é demonstrado os campos (colunas) e sua respectiva descrição a que se refere tal dado em cada tabela construída no banco de dados.</p>

<h4 style="text-align:left">TB_JORNAIS</h4>
<p align='justify'>Tabela com informações básicas do jornal.</p>
    
|Campo|Descrição|
|:----|:--------|
|ID_JORNAL|Identificador único do jornal|
|DESC_JORNAL|Nome do jornal|
|URL_JORNAL|URL principal do jornal|

<h4 style="text-align:left">TB_URL_NOTICIAS</h4>
<p align='justify'>Tabela com informações das URL's da notícias extraídas do jornal.</p>

|Campo|Descrição|
|:----|:--------|
|ID_NOTICIA|Identificador único da notícia|
|DATA_EXTRACAO|Data da extração da url da notícia|
|DESC_JORNAL|Nome do jornal|
|DESC_TEMA|Nome do tema da notícia|
|URL_NOTICIA|URL da notícia|
|FLAG_EXTRAIDA|Flag (True ou False) se já foi extraído o texto da notícia|

<h4 style="text-align:left">TB_TEXTO_NOTICIAS</h4>
<p align='justify'>Tabela com informações das notícias extraídas do jornal.</p>

|Campo|Descrição|
|:----|:--------|
|ID_NOTICIA|Identificador único da notícia|
|URL|URL da notícia|
|DATA_PUBLICACAO|Data e hora da publicação da notícia|
|AUTOR|Autor(es) da notícia|
|TITULO|Título (manchete) da notícia|
|TEXTO|Texto da notícia|

<h2 style="text-align:left">Entendimento dos Dados</h2>

<p align='justify'>Nesta etapa é realizada a exploração dos dados, com o objetivo de entender as características da base de dados e possíveis ajustes necessários a serem realizados antes da etapa de visualização.</p>

In [1]:
#!pip install wordcloud -q

In [2]:
# Importa Bibliotecas.
import os
import sys
from dotenv import load_dotenv
import time
import re
from datetime import date

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# Apresenta os dados no notebook
%matplotlib inline

import seaborn as sns
# Estilo dos gráficos
sns.set_style("darkgrid")

from wordcloud import WordCloud

import psycopg2 
import sqlalchemy               
from sqlalchemy import create_engine
from sqlalchemy import text

load_dotenv()

# Parametros do Pandas, limitando a quantidade máxima e a largura das colunas.
pd.set_option('max_colwidth', 5000)

In [3]:
# Credenciais banco
host=os.getenv('BD_POSTGRESQL_HOST')
port=os.getenv('BD_POSTGRESQL_PORT')
dbname=os.getenv('BD_POSTGRESQL_DBNAME')
user=os.getenv('BD_POSTGRESQL_USE')
password=os.getenv('BD_POSTGRESQL_PASSWORD')

In [4]:
path = "C:/Users/Usuario/Documents/GitHub/monitoring-webscraping-newspaper/dados/"

<h3 style="text-align:left">TB_JORNAIS</h3>

In [5]:
# Consultando dados anteriores no BD.
engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')
conn = engine.connect()

df_jornais = pd.read_sql(text('''select * from tb_jornais;'''), conn)

# Proporção dos dados em linhas e colunas
print(df_jornais.shape)

conn.close()

(1, 3)


In [6]:
# Verificando os metadados da base
df_jornais.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID_JORNAL    1 non-null      int64 
 1   DESC_JORNAL  1 non-null      object
 2   URL_JORNAL   1 non-null      object
dtypes: int64(1), object(2)
memory usage: 152.0+ bytes


In [7]:
# Verificando os tipos de dados.
print("Tipos de dados e sua frequência\n{}".format(df_jornais.dtypes.value_counts()))

Tipos de dados e sua frequência
object    2
int64     1
dtype: int64


In [8]:
# Observando a abse de dados
df_jornais.head()

,ID_JORNAL,DESC_JORNAL,URL_JORNAL
0,1,Correio Braziliense,https://www.correiobraziliense.com.br/


In [9]:
# Verificação da existência de dados ausentes/faltantes
df_jornais.isnull().sum()

ID_JORNAL      0
DESC_JORNAL    0
URL_JORNAL     0
dtype: int64

In [10]:
# Verificando dados nulos por registro e retornando a maior quantidade.
df_jornais.isna().sum(axis=1).max()

0

<h3 style="text-align:left">TB_URL_NOTICIAS</h3>

In [11]:
# Consultando dados anteriores no BD.
engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')
conn = engine.connect()

df_url = pd.read_sql(text('''select * from tb_url_noticias;'''), conn)

# Proporção dos dados em linhas e colunas
print(df_url.shape)

conn.close()

(18839, 6)


In [12]:
# Verificando os metadados da base
df_url.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18839 entries, 0 to 18838
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID_NOTICIA     18839 non-null  int64 
 1   DATA_EXTRACAO  18839 non-null  object
 2   DESC_JORNAL    18839 non-null  object
 3   DESC_TEMA      18839 non-null  object
 4   URL_NOTICIA    18839 non-null  object
 5   FLAG_EXTRAIDA  18839 non-null  bool  
dtypes: bool(1), int64(1), object(4)
memory usage: 754.4+ KB


In [13]:
# Verificando os tipos de dados.
print("Tipos de dados e sua frequência\n{}".format(df_url.dtypes.value_counts()))

Tipos de dados e sua frequência
object    4
int64     1
bool      1
dtype: int64


In [14]:
# Observando a abse de dados
df_url.head()

,ID_NOTICIA,DATA_EXTRACAO,DESC_JORNAL,DESC_TEMA,URL_NOTICIA,FLAG_EXTRAIDA
0,3,2022-09-24,correiobraziliense,home,https://www.correiobraziliense.com.br/politica/2022/09/5039139-tse-identifica-rs-605-milhoes-em-doacoes-suspeitas.html,True
1,4,2022-09-24,correiobraziliense,home,https://www.correiobraziliense.com.br/cidades-df/2022/09/5039108-confira-a-agenda-dos-candidatos-ao-gdf-para-este-sabado.html,True
2,261,2022-10-05,correiobraziliense,opiniao,https://www.correiobraziliense.com.br//opiniao/2022/10/5041502-artigo-o-brasil-mudou.html,True
3,1639,2022-10-13,correiobraziliense,home,https://www.correiobraziliense.com.br/cbforum/testedopezinho.html,True
4,2156,2022-10-17,correiobraziliense,tecnologia,https://www.correiobraziliense.com.br/tecnologia/2022/10/5044373-ia-vence-os-obstaculos.html,True


In [15]:
# Verificação da existência de dados ausentes/faltantes
df_url.isnull().sum()

ID_NOTICIA       0
DATA_EXTRACAO    0
DESC_JORNAL      0
DESC_TEMA        0
URL_NOTICIA      0
FLAG_EXTRAIDA    0
dtype: int64

In [16]:
# Verificando dados nulos por registro e retornando a maior quantidade.
df_url.isna().sum(axis=1).max()

0

<h3 style="text-align:left">TB_TEXTO_NOTICIAS</h3>

In [17]:
# Consultando dados anteriores no BD.
engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')
conn = engine.connect()

df_txt = pd.read_sql(text('''select * from tb_texto_noticias;'''), conn)
print(df_txt.shape)

conn.close()

(18839, 6)


In [18]:
# Verificando os metadados da base
df_txt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18839 entries, 0 to 18838
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID_NOTICIA       18839 non-null  int64 
 1   URL              18839 non-null  object
 2   DATA_PUBLICACAO  18546 non-null  object
 3   AUTOR            18839 non-null  object
 4   TITULO           18839 non-null  object
 5   TEXTO            18839 non-null  object
dtypes: int64(1), object(5)
memory usage: 883.2+ KB


In [19]:
# Verificando os tipos de dados.
print("Tipos de dados e sua frequência\n{}".format(df_txt.dtypes.value_counts()))

Tipos de dados e sua frequência
object    5
int64     1
dtype: int64


In [20]:
# Observando a abse de dados
df_txt.head(2)

ID_NOTICIA  \
0           1   
1           2   

                                                                                                                                          URL  \
0  https://www.correiobraziliense.com.br/politica/2022/09/5039152-bolsonaro-e-lula-jogam-as-cartas-para-reta-final-da-campanha-eleitoral.html   
1    https://www.correiobraziliense.com.br/politica/2022/09/5039144-disputa-pelo-voto-entre-lula-e-bolsonaro-divide-igrejas-pentecostais.html   

               DATA_PUBLICACAO                                AUTOR  \
0  2022-09-24-0303:55:00-10800  {"'Ingrid Soares, Victor Correia'"}   
1  2022-09-24-0303:55:00-10800            {"'Correio Braziliense'"}   

                                                                   TITULO  \
0  Bolsonaro e Lula jogam as cartas para reta final da campanha eleitoral   
1    Disputa pelo voto entre Lula e Bolsonaro divide igrejas pentecostais   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [21]:
# Verificação da existência de dados ausentes/faltantes
df_txt.isnull().sum()

ID_NOTICIA           0
URL                  0
DATA_PUBLICACAO    293
AUTOR                0
TITULO               0
TEXTO                0
dtype: int64

In [51]:
# Verificando dados nulos por registro e retornando a maior quantidade.
df_txt.isna().sum(axis=1).value_counts()

0    18546
1      293
dtype: int64

<h2 style="text-align:left">Preparação dos Dados</h2>
<p align='justify'>Nesta etapa é realizada a preparação dos dados, com o objetivo de ajustar as inconsistências encontradas durante a análise exploratória dos dados, permitindo que na etapa de visualização dos dados, os mesmos estejam de forma adequada a apresentação das informações.</p>

In [23]:
# file_name = "tb_texto_noticias"
# extensao = ".txt"

# df_txt.to_csv(path+file_name+"_tab"+"_utf8"+extensao, sep='\t', encoding='utf-8')
# df_txt.to_csv(path+file_name+extensao, sep='|', encoding='utf-8', index=False)
# df_txt.to_csv(path+file_name+"_bardup"+"_utf8"+extensao, sep=';', encoding='utf-8')

# df_txt.to_csv(path+file_name+"_tab"+"_latin1"+extensao, sep='\t', encoding ='latin1')
# df_txt.to_csv(path+file_name+extensao, sep='|', encoding ='not utf-8', index=False)
# df_txt.to_csv(path+file_name+"_bardup"+"_latin1"+extensao, sep=';', encoding ='latin1')